#Language Model

문장에 확률을 할당하는 모델\
통계 vs AI\
이전 단어들이 주어졌을 때 다음 단어를 예측\
주어진 양쪽의 단어들로부터 가운데 비어있는 단어를 예측\
Language Modeling : 현재 가지고있는 시퀀스로 부터 모르는 단어를 찾아내는 것\
기계 번역(Machine Translation)\
오타 교정(Spell Correction)\
음성 인식(Speech Recognition)


## SLM(Statistical Language Model) 통계적 언어 모델

###N-gram

unigram\
bigram\
trigram\
4-gram\
gram이 많아지면 성능이 올라가지만, 희소성의 문제가 다시 발생함\
통상적으로는 5-gram까지 사용

##한국어 모델

### 언어 모델 성능 평가 PPL(Perplexity)

#단어표현 Word Representation

##Bag of Words

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.6 MB/s 
     |████████████████████████████████| 453 kB 43.1 MB/s 


In [2]:
from konlpy.tag import Okt
okt = Okt()
def build_bag_of_words(document):
  document = document.replace('.'," ") # 마침표를 공백으로 변환
  tokenized_document = okt.morphs(document) #텍스트를 형태소 단위로 나눔
  word_to_index = {}
  bow = []
  for word in tokenized_document:
    if word not in word_to_index.keys():
      word_to_index[word] = len(word_to_index) # word_to_index에 없는 형태소라면 딕셔너리의 마지막 순서에 단어 추가
      bow.insert(len(word_to_index)-1, 1) # insert(a,b) a위치에 b값을 추가/ 해당 형태소의 인덱스에 1 추가하기
    else:
      index = word_to_index.get(word) # .get() 메서드 : 해당 키값에 해당하는 밸류값을 가져옴/ index는 여기서 형태소의 인덱스를 나타냄
      bow[index] = bow[index] + 1 # 해당 인덱스 번째에 1추가
  return word_to_index, bow

In [3]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print("vocabulary :", vocab)
print("bag of words vector :", bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


In [4]:
doc2 = '소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.'
vocab, bow = build_bag_of_words(doc2)
print("vocabulary :", vocab)
print("bag of words vector :", bow)

vocabulary : {'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}
bag of words vector : [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]


In [5]:
# 서로 다른 두문장에서 겹치는 형태소를 고려하기 위해, 문장을 합침
doc3 = doc1 + " " + doc2 
vocab, bow = build_bag_of_words(doc3)
print("vocabulary :", vocab)
print("bag of words vector :", bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
bag of words vector : [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


###사이킷런 이용하기

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ["you know I want your love. because I love you."] # CountVectorizer() 클래스를 사용하기 위해서는 리스트화를 시켜야함
vector = CountVectorizer()
print("bag of words vector :", vector.fit_transform(corpus).toarray())
print("vocabulary :", vector.vocabulary_) #print를 이용하면 인덱스 순이 아니라, 먼저나온 단어 순서임
"vocabulary :", vector.vocabulary_

bag of words vector : [[1 1 2 1 2 1]]
vocabulary : {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


('vocabulary :',
 {'because': 0, 'know': 1, 'love': 2, 'want': 3, 'you': 4, 'your': 5})

In [9]:
corpus = [doc1] 
vector = CountVectorizer()
print("bag of words vector :", vector.fit_transform(corpus).toarray())
vector.vocabulary_
# 형태소로 구분이 되는 것이 아니라 여백이 기준이라 영어에 적합한 모델임

bag of words vector : [[1 1 1 1 1 1 1]]


{'느끼는': 0, '다르다': 1, '물가상승률과': 2, '물가상승률은': 3, '발표하는': 4, '소비자가': 5, '정부가': 6}

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words = ["the","a","an","is","not"]) # 매개변수로 불용어리스트를 설정해주면, 그 불용어를 제외한 단어들로 BOW를 만들어줌
print("bag of words vector :", vect.fit_transform(text).toarray())
print("vocabulary :", vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [12]:
vect = CountVectorizer(stop_words = "english") # stop_words 매개변수에 리스트로 입력한게 아니므로, 클래스에 저장돼있는 "영어" 불용어 리스트를 가져온다는 뜻임
print("bag of words vector :", vect.fit_transform(text).toarray())
print("vocabulary :", vect.vocabulary_)
# it, everything 도 영어 불용어 리스트에 포함 되있다는것을 알 수 있음

bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


In [13]:
import nltk
nltk.download("stopwords")
nltk_stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words = nltk_stop_words)
print("bag of words vector :", vect.fit_transform(text).toarray())
print("vocabulary :", vect.vocabulary_)
# nltk의 불용어 리스트에는 everything이 없음

bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Document Term Matrix

다수의 document에 등장하는 각 단어들의 빈도를 행렬로 표현한 것

###TF-IDF

Term Frequency-Inverse Document Frequency\
DTM 내의 각 단어들마다 중요한 정도를 가중치 (DTM -> TF-IDF 가중치 부여)\
문서의 유사도, 검색 결과의 중요도, 특정 단어의 중요도\
TF-IDF: TF*IDF(document: d, term: t, total number of document: n)\
tf(d,t):특정 문서 d에서의 특정 단어 t의 등장 횟수\
df(t):특정 단어 t가 등장한 문서의 수.\
idf(d,t):df(t)에 반비례하는 수\
idf(d,t) = log(n/(1+df(t))) 



In [14]:
import pandas as pd
from math import log

docs = [
        "먹고 싶은 사과",
        "먹고 싶은 바나나",
        "길고 노란 바나나 바나나",
        "저는 과일이 좋아요"
]
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [21]:
vocab1 = set()
for doc in docs:
  for w in doc.split():
    vocab1.add(w)
print(vocab) # 내림차순
print()
print(vocab1) #빈도순

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

{'바나나', '먹고', '싶은', '사과', '노란', '길고', '과일이', '좋아요', '저는'}


In [26]:
N = len(docs)
# tf(d,t):특정 문서 d에서의 특정 단어 t의 등장 횟수
def tf(t,d):
  return d.count(t)

# idf(d,t):df(t)에 반비례하는 수
def idf(t):
  df = 0
  for doc in docs:
    df += t in doc # t in doc 가 True 인경우 1로 반환/ df는 해당 단어가 포함된 문장의 수
  return log(N/(df+1))


def tfidf(t,d):
  return tf(t,d)*idf(t)

In [27]:
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
  result.append([]) # 매번 result안에 빈 리스트 생성

  d = docs[i] #각 문장을 d라고 지정
  for j in range(len(vocab)):
    t = vocab[j] # 각 단어들을 t라고 지정
    result[-1].append(tf(t,d)) # 첫번째 for 문에서 생성했던 result안의 빈 리스트에 tf(t,d) 입력 (d에서 t의 등장 횟수)
tf_ = pd.DataFrame(result, columns = vocab)

In [28]:
print(result)

[[0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 0, 1, 1, 0, 1, 0, 0], [0, 1, 1, 0, 2, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 1]]


In [29]:
tf_

과일이  길고  노란  먹고  바나나  사과  싶은  저는  좋아요
0    0   0   0   1    0   1   1   0    0
1    0   0   0   1    1   0   1   0    0
2    0   1   1   0    2   0   0   0    0
3    1   0   0   0    0   0   0   1    1

In [31]:
result = []
for j in range(len(vocab)):
  t = vocab[j]
  result.append(idf(t))
idf_=pd.DataFrame(result, index = vocab, columns = ["IDF"])
print(result)
idf_
# idf는 단어가 출현한 문장이 많을 수록 작음

[0.6931471805599453, 0.6931471805599453, 0.6931471805599453, 0.28768207245178085, 0.28768207245178085, 0.6931471805599453, 0.28768207245178085, 0.6931471805599453, 0.6931471805599453]


IDF
과일이  0.693147
길고   0.693147
노란   0.693147
먹고   0.287682
바나나  0.287682
사과   0.693147
싶은   0.287682
저는   0.693147
좋아요  0.693147

In [32]:
result = []
for i in range(N):
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tfidf(t,d))
tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

과일이        길고        노란        먹고       바나나        사과        싶은  \
0  0.000000  0.000000  0.000000  0.287682  0.000000  0.693147  0.287682   
1  0.000000  0.000000  0.000000  0.287682  0.287682  0.000000  0.287682   
2  0.000000  0.693147  0.693147  0.000000  0.575364  0.000000  0.000000   
3  0.693147  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

         저는       좋아요  
0  0.000000  0.000000  
1  0.000000  0.000000  
2  0.000000  0.000000  
3  0.693147  0.693147

사이킷런 활용 실습

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
          "you know I want your love",
          "I like you",
          "what should I do"
]
tfidfv = TfidfVectorizer()
# 코퍼러스로부터 각 단어의 빈도수를 기록
print(tfidfv.fit_transform(corpus).toarray())
# 각 단어와 맵핑된 인덱스 출력
print(tfidfv.vocabulary_)

# keras에서도 가능

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


### 문서 유사도

In [45]:
# 문서1 : 저는 사과 좋아요
# 문서2 : 저는 바나나 좋아요
# 문서3 : 저는 바나나 좋아요 저는 바나나 좋아요

import numpy as np
from numpy import dot
from numpy.linalg import norm
def cos_sim(A,B):
  return dot(A,B)/(norm(A)*norm(B)) # norm()은 유클리드 거리를 계산해줌
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])
print("문서1과 문서2의 유사도:",cos_sim(doc1,doc2))
print("문서1과 문서3의 유사도:",cos_sim(doc1,doc3))
print("문서2와 문서3의 유사도:",cos_sim(doc2,doc3)) # 카운트 면에서는 다를 수 있지만, 방향성은 일치함

문서1과 문서2의 유사도: 0.6666666666666667
문서1과 문서3의 유사도: 0.6666666666666667
문서2와 문서3의 유사도: 1.0000000000000002


영화 추천 시스템

In [46]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# 파일을 읽고 상위 2개 행만 보기
df = pd.read_csv("movies_metadata.csv")
df.head(2)
# 상위 2만개의 샘플을 data에 저장
data = df.head(20000)
# overview 열에 존재하는 모든 결측값 수 확인
data["overview"].isnull().sum()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


135

In [47]:
# 결측값을 빈 값으로 대체
data["overview"] = data["overview"].fillna("")
tfidf = TfidfVectorizer(stop_words="english")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
tfidf_matrix = tfidf.fit_transform(data["overview"])
print("TF-IDF 행렬의 크기(shape):", tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape): (20000, 47487)


In [49]:
# 20,000개의 문서 벡터에 대해서 상호 간의 코사인 유사도 확인
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("코사인 유사도 연산 결과:", cosine_sim.shape)

코사인 유사도 연산 결과: (20000, 20000)


In [50]:
data.head()

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   
4                                   NaN  11862  tt0113041                en   

                original_title  \
0                    Toy Story   
1                      Jumanji   
2             Grumpier Old Men   
3            Waiting to Exhale   
4  Father of the Bride Part II   

                                            overview  ... release_date  \
0  Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1  When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2  A family wedding reignites the ancient feud be...  ...   1995-12-22   
3  Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4  Just when George Banks has recovered from his ...  ...   1995-02-10   

       revenue runtime                                   spoken_languages  \
0  373554033.0    81.0           [{'iso_639_1': 'en', 'name': 'English'}]   
1  262797249.0   104.0  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
2          0.0   101.0           [{'iso_639_1': 'en', 'name': 'English'}]   
3   81452156.0   127.0           [{'iso_639_1': 'en', 'name': 'English'}]   
4   76578911.0   106.0           [{'iso_639_1': 'en', 'name': 'English'}]   

     status                                            tagline  \
0  Released                                                NaN   
1  Released          Roll the dice and unleash the excitement!   
2  Released  Still Yelling. Still Fighting. Still Ready for...   
3  Released  Friends are the people who let you be yourself...   
4  Released  Just When His World Is Back To Normal... He's ...   

                         title  video vote_average vote_count  
0                    Toy Story  False          7.7     5415.0  
1                      Jumanji  False          6.9     2413.0  
2             Grumpier Old Men  False          6.5       92.0  
3            Waiting to Exhale  False          6.1       34.0  
4  Father of the Bride Part II  False          5.7      173.0  

[5 rows x 24 columns]

In [63]:
def get_recommendations(title, sim = cosine_sim):
# 영화의 타이틀을 사용해 해당 영화의 인덱스를 받아온다.
  idx = data.index[(data["title"]==title)]
  idx = idx[0]
# 해당 영화와 모든 영화와의 유사도를 가져온다.
  sim_scores = list(enumerate(cosine_sim[idx]))
# 유사도에 따라 영화들을 정렬한다.
  sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
# 가장 유사한 10개의 영화를 받아온다.
  sim_scores = sim_scores[1:11]
# 가장 유사한 10개의 영화의 인덱스를 얻는다.
  movie_ind = [idx[0] for idx in sim_scores]
# 가장 유사한 10개의 영화의 제목을 리턴한다.
  return data["title"].iloc[movie_ind]

In [64]:
get_recommendations("Toy Story")

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
485                      Malice
11606              Factory Girl
Name: title, dtype: object

Jaccard Similarity

In [66]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print('문서1 :',tokenized_doc1)
print('문서2 :',tokenized_doc2)
union = set(tokenized_doc1)|(set(tokenized_doc2))
intersection = set(tokenized_doc1)&(set(tokenized_doc2))
print('자카드 유사도 :',len(intersection)/len(union)) # 교집합/합집합

문서1 : ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서2 : ['apple', 'banana', 'coupon', 'passport', 'love', 'you']
자카드 유사도 : 0.16666666666666666


Manhattan Similarity(L1 distance)